### **Covertype Neuronska mreža**

Cilj projekta jeste da se istrenira neuronska mreža nad Covertype setom podataka.
Covertype dataset sadrži podatke o tipu šumskog pokrivača u Roosvelt National Forest. Svaki record predstavlja parcelu površine 30x30m, gdje za svaku parcelu imamo njene osobine kao npr. nagib, udaljenost od izvora vode, tipovi tla i sl. Imamo ukupno 54 osobine tj. atributa.  
Cilj je da se na osnovu ovih osobina zaključi kakav će se tip šumskog pokrivača razviti na datoj parceli, pri čemu imamo 7 mogućih tipova: Spruce/Fir, Lodgepole Pine, Ponderosa Pine, Cottonwood/Willow, Aspen, Douglas-fir i Krummholz.

Učitavanje potrebnih biblioteka:

In [48]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.metrics import accuracy_score

Učitavanje podataka:

In [49]:
dataset=fetch_covtype()
X=dataset.data
y=dataset.target
print(X.shape)
print(y.shape)

(581012, 54)
(581012,)


Vidimo da dataset ima 581012 zapisa.


Podjela na trening i testni skup:

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

Konvertovanje u tenzore:

In [51]:
X_train_t = T.tensor(X_train).float()
X_test_t = T.tensor(X_test).float()
y_train_t = T.LongTensor(y_train)
y_test_t = T.LongTensor(y_test)

Definišemo trening i testni dataset na osnovu tenzora.

In [52]:
train_dataset=TensorDataset(X_train_t, y_train_t)
test_dataset=TensorDataset(X_test_t, y_test_t)

DataLoader klasa omogućava jednostavno učitavanje podataka u batchu na osnovu definisanog dataseta.

In [53]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

Definicija modela; Model ima 54 ulaza koji odgovaraju broju atributa uzoraka i 7 izlaza koji određuju klasu.

In [54]:
class CovertypeClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1=nn.Linear(54,64)
    self.act1=nn.ReLU()
    self.layer2=nn.Linear(64, 128)
    self.act2=nn.ReLU()
    self.drop2=nn.Dropout(p=0.2)
    self.layer3=nn.Linear(128, 128)
    self.act3=nn.ReLU()
    self.drop3=nn.Dropout(p=0.2)
    self.output=nn.Linear(128, 54)

  def forward(self, x):
    x=self.act1(self.layer1(x))
    x=self.act2(self.layer2(x))
    x=self.drop2(x)
    x=self.act3(self.layer3(x))
    x=self.drop3(x)
    x=self.output(x)
    return x

Poznato je da Pytorch omogućava izvršavanje koda na grafičkoj kartici što značajno ubrzava treniranje mreže. Kako bismo omogućili datu funkcionalnost potrebno je definisati uređaj na kom će se izvršavati sve operacije. Nakon definisanja uređaja potrebno je i mrežu i sve podatke prebaciti na odgovarajući uređaj. Ukoliko se podaci ili mreža nalaze na različitim uređajima (jedan na CPU drugi na GPU) Pytorch ce javiti grešku.

In [55]:
device = T.device("cuda:0") if T.cuda.is_available() else T.device("cpu") # koristimo gpu samo ako je dostupan, inace koristimo cpu
print(device)

cpu


In [56]:
net=CovertypeClassifier().to(device)

Metoda za testiranje mreže:

In [57]:
def test_model (net, test_loader):
  total_loss = 0
  correct = 0
  total_samples = 0
  net.eval()

  with T.no_grad():
   for x, y in test_loader:
     x=x.to(device)
     y=y.to(device)
     preds = net(x)
     _, predicted_labels = T.max(preds.data, 1)
     loss=loss_fn(preds, y)
     total_loss += loss.item()
     correct += (predicted_labels == y).sum().item()
     total_samples += len(y)

  accuracy = correct/total_samples
  average_loss=total_loss/total_samples

  print("Tacnost: ", accuracy)
  print("Prosjecan gubitak: ", average_loss)
  print(total_loss / len(test_dataset))

In [58]:
optimizer=T.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
loss_fn = nn.CrossEntropyLoss()

In [59]:
test_model(net,test_loader)

Tacnost:  0.0
Prosjecan gubitak:  2.4712931429454588
2.4712931429454588


In [60]:
EPOCHS = 50

Treniranje modela.

In [61]:
net.train()
for i in range(EPOCHS):
    if i % 10 == 0:
        print(f"Current epoch: {i}")

    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        preds = net(x)
        loss = loss_fn(preds, y)
        loss.backward()
        print("Loss:", loss.detach().cpu().numpy())
        break
        optimizer.step()


Current epoch: 0
Loss: 95.84341
Loss: 91.9983
Loss: 98.66812
Loss: 91.88773
Loss: 89.987076
Loss: 81.80029
Loss: 88.01677
Loss: 89.29829
Loss: 90.81656
Loss: 92.25628
Current epoch: 10
Loss: 87.22506
Loss: 84.46403
Loss: 94.85898
Loss: 89.770226
Loss: 98.132164
Loss: 91.42642
Loss: 85.86803
Loss: 85.34506
Loss: 88.570564
Loss: 91.194214
Current epoch: 20
Loss: 82.18721
Loss: 79.38918
Loss: 82.60354
Loss: 90.136955
Loss: 80.01209
Loss: 80.376045
Loss: 79.20615
Loss: 91.617546
Loss: 88.98636
Loss: 90.68799
Current epoch: 30
Loss: 94.20181
Loss: 83.52865
Loss: 88.660645
Loss: 88.53902
Loss: 82.37291
Loss: 94.85188
Loss: 73.35189
Loss: 104.65558
Loss: 88.67385
Loss: 92.631836
Current epoch: 40
Loss: 87.50375
Loss: 84.46471
Loss: 84.50464
Loss: 92.29078
Loss: 88.91976
Loss: 86.337204
Loss: 90.79738
Loss: 92.42507
Loss: 97.53711
Loss: 87.142044


In [62]:
test_model(net,test_loader)

Tacnost:  0.0
Prosjecan gubitak:  2.4712931429454588
2.4712931429454588


Definisanje validacionog skupa podataka i implementacija early stopping u toku treninga
tako da se trening prekine kada greška nad validacionim skupom počne da raste. Tolerišu se 2
epohe rasta greške (patience = 2), a nakon toga se prekida trening ukoliko se greška poveća i u trećoj
uzastopnoj epohi.

In [63]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=42)

In [64]:
X_val_t = T.tensor(X_val).float()
y_val_t = T.LongTensor(y_val)

In [65]:
val_dataset = TensorDataset(X_val_t, y_val_t)
val_loader = DataLoader(val_dataset, batch_size=32)

In [66]:
def calculate_validation_loss(model, loss_fn, val_loader):
  total_loss=0
  total_samples=0

  model.eval()

  with T.no_grad():
    for x_val, y_val in val_loader:
      x_val = x_val.to(device)
      y_val = y_val.to(device)

      preds = model(x_val)
      loss = loss_fn(preds, y_val)
      total_loss += loss.item() * len(x_val)
      total_samples += len(x_val)

  average_loss = total_loss/total_samples
  return average_loss

In [67]:
net.train()
best_loss=float('inf')
patience=2
early_stopping_counter=0

for epoch in range(EPOCHS):
  if epoch % 10 == 0:
    print (f"Current epoch: {epoch}")

    #Treniranje modela
    val_loss = calculate_validation_loss(net, loss_fn, val_loader)

    if val_loss<best_loss:
      best_loss = val_loss
      early_stopping_counter=0
    else:
      early_stopping_counter+=1

    if early_stopping_counter >= patience:
      print ("Early stopping triggered. Training stopped. ")
      break

  for x, y in train_loader:
    x = x.to(device)
    y = y.to(device)
    preds = net(x)
    loss=loss_fn(preds, y)
    optimizer.zero_grad()
    loss.backward()
    print("Loss:", loss.detach().cpu().numpy())
    break
    optimizer.step()

Current epoch: 0
Loss: 88.34089
Loss: 79.21117
Loss: 74.8638
Loss: 82.77491
Loss: 86.38067
Loss: 85.11609
Loss: 80.673325
Loss: 78.84601
Loss: 77.53549
Loss: 79.06378
Current epoch: 10
Loss: 75.68697
Loss: 78.92473
Loss: 73.993744
Loss: 76.842705
Loss: 74.2341
Loss: 77.680145
Loss: 85.252945
Loss: 80.05445
Loss: 82.1998
Loss: 75.257935
Current epoch: 20
Early stopping triggered. Training stopped. 


Pretraga hiperparametara

In [68]:
learning_rates = [0.01, 0.1]
hidden_sizes = [64, 128]
dropouts = [0.2, 0.5]

In [69]:
best_accuracy = 0.0
best_model = None

for learning_rate in learning_rates:
  for hidden_size in hidden_sizes:
    for dropout in dropouts:
      net =  CovertypeClassifier()
      optimizer = optim.Adam(net.parameters(), lr=learning_rate)
      loss_fn = nn.CrossEntropyLoss()


      net.train()
      for epoch in range(EPOCHS):
        if epoch % 5==0:
          print (f"Current epoch: {epoch}")

          for x,y in train_loader:
            x = x.to(device)
            y = y.to(device)
            preds = net(x)
            loss=loss_fn(preds, y)
            optimizer.zero_grad()
            loss.backward()
            print("Loss", loss.detach().cpu().numpy())
            break
            optimizer.step()


      net.eval()
      with T.no_grad():
        val_predictions = []
        val_targets = []
        for x_val, y_val in val_loader:
          x_val = x_val.to(device)
          y_val = y_val.to(device)

          preds=net(x_val)
          val_predictions.extend(T.argmax(preds, dim=1).tolist())
          val_targets.extend(y_val.tolist())

      val_predictions = np.array(val_predictions)
      val_targets = np.array(val_targets)
      validation_accuracy = accuracy_score(val_targets, val_predictions)

      if validation_accuracy > best_accuracy:
        best_accuracy = validation_accuracy
        best_model = net.state_dict()

Current epoch: 0
Loss 75.7333
Current epoch: 5
Loss 74.17928
Current epoch: 10
Loss 65.243126
Current epoch: 15
Loss 76.57684
Current epoch: 20
Loss 78.04378
Current epoch: 25
Loss 73.968376
Current epoch: 30
Loss 75.87592
Current epoch: 35
Loss 70.57671
Current epoch: 40
Loss 78.30932
Current epoch: 45
Loss 73.09948
Current epoch: 0
Loss 97.51761
Current epoch: 5
Loss 94.60262
Current epoch: 10
Loss 99.20025
Current epoch: 15
Loss 99.30196
Current epoch: 20
Loss 113.12736
Current epoch: 25
Loss 97.67052
Current epoch: 30
Loss 92.750885
Current epoch: 35
Loss 98.86315
Current epoch: 40
Loss 105.38454
Current epoch: 45
Loss 88.23423
Current epoch: 0
Loss 45.647327
Current epoch: 5
Loss 60.728283
Current epoch: 10
Loss 43.275616
Current epoch: 15
Loss 42.559868
Current epoch: 20
Loss 47.64454
Current epoch: 25
Loss 59.59527
Current epoch: 30
Loss 48.10307
Current epoch: 35
Loss 45.79082
Current epoch: 40
Loss 52.41893
Current epoch: 45
Loss 50.639267
Current epoch: 0
Loss 65.145256
Curre

In [70]:
net.load_state_dict(best_model)
net.eval()
with T.no_grad():
    test_predictions = []
    test_targets = []
    for x_test, y_test in test_loader:
        x_test = x_test.to(device)
        y_test = y_test.to(device)

        preds = net(x_test)
        test_predictions.extend(T.argmax(preds, dim=1).tolist())
        test_targets.extend(y_test.tolist())


test_predictions = np.array(test_predictions)
test_targets = np.array(test_targets)
test_accuracy = accuracy_score(test_targets, test_predictions)

print (f"Najbolja preciznost na validacionom skupu: {test_accuracy}")
print(f"Learning Rate: {learning_rate}, Dropout: {dropout}, Validation Accuracy: {validation_accuracy} ")

Najbolja preciznost na validacionom skupu: 0.09572902592876259
Learning Rate: 0.1, Dropout: 0.5, Validation Accuracy: 0.0 
